## Group Information: To be filled by the candidates.

### Group Number: 14
### Members Roll Numbers: 24AI60R13, 24AI60R46
### Members Name: Vinayak Mali, Saurabh Jaiswal



## 1. Pre-requist

In [ ]:
!pip install pykeen

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm.notebook import tqdm
from pykeen.datasets import Nations, Kinships

INFO:pykeen.utils:Using opt_einsum


In [3]:
print("Set random seeds")
torch.manual_seed(0)
np.random.seed(0)

Set random seeds


In [4]:
print("device check")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

device check
Using device: cuda


## 2. Dataset Loading

In [5]:
# Load Nations and Kinships datasets
print("Loading datasets...")
nations_dataset = Nations()
kinships_dataset = Kinships()

Loading datasets...


In [6]:
print("Extracting training, validation, and test triples for both datasets")
nations_train = nations_dataset.training.mapped_triples.tolist()
nations_test = nations_dataset.testing.mapped_triples.tolist()
kinships_train = kinships_dataset.training.mapped_triples.tolist()
kinships_test = kinships_dataset.testing.mapped_triples.tolist()

Extracting training, validation, and test triples for both datasets


In [7]:
print("Getting entity and relation counts")
nations_num_entities = nations_dataset.num_entities
nations_num_relations = nations_dataset.num_relations
kinships_num_entities = kinships_dataset.num_entities
kinships_num_relations = kinships_dataset.num_relations

Getting entity and relation counts


In [8]:
print(f"Nations Dataset - Entities: {nations_num_entities}, Relations: {nations_num_relations}")
print(f"Kinships Dataset - Entities: {kinships_num_entities}, Relations: {kinships_num_relations}")

Nations Dataset - Entities: 14, Relations: 55
Kinships Dataset - Entities: 104, Relations: 25


##3. Triples

In [9]:
print("Function for extracting triples from datasets")
def extract_triples(triples):
    return [(h, r, t) for h, r, t in triples]

Function for extracting triples from datasets


In [10]:
print("Extracting triples from Nations and Kinships datasets")
nations_train_triples = extract_triples(nations_train)
nations_test_triples = extract_triples(nations_test)
kinships_train_triples = extract_triples(kinships_train)
kinships_test_triples = extract_triples(kinships_test)

Extracting triples from Nations and Kinships datasets


In [11]:
print("Data preprocessing and triple extraction complete.")

Data preprocessing and triple extraction complete.


## 4. TransE & transR


In [12]:
print("Defining the TransE model class")
class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim, margin, norm=1):
        super(TransE, self).__init__()
        self.embedding_dim = embedding_dim
        self.margin = margin
        self.norm = norm
        # Initialize entity and relation embeddings
        self.entity_embeddings = nn.Embedding(num_entities, embedding_dim)
        self.relation_embeddings = nn.Embedding(num_relations, embedding_dim)
        # Initialize embeddings
        nn.init.uniform_(self.entity_embeddings.weight.data, -6/np.sqrt(embedding_dim), 6/np.sqrt(embedding_dim))
        nn.init.uniform_(self.relation_embeddings.weight.data, -6/np.sqrt(embedding_dim), 6/np.sqrt(embedding_dim))

    def forward(self, head, relation, tail):
        # Get embeddings for head, relation, and tail
        head_emb = self.entity_embeddings(head)
        rel_emb = self.relation_embeddings(relation)
        tail_emb = self.entity_embeddings(tail)

        # Calculate score based on the TransE distance function
        score = torch.norm(head_emb + rel_emb - tail_emb, p=self.norm, dim=1)
        return score

    def get_embeddings(self):
        # Return entity and relation embeddings
        return self.entity_embeddings.weight, self.relation_embeddings.weight


Defining the TransE model class


In [13]:
print("Defining the TransR model class")
class TransR(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim, margin, rel_embedding_dim):
        super(TransR, self).__init__()
        self.embedding_dim = embedding_dim
        self.rel_embedding_dim = rel_embedding_dim
        self.margin = margin
        # Initialize entity and relation embeddings
        self.entity_embeddings = nn.Embedding(num_entities, embedding_dim)
        self.relation_embeddings = nn.Embedding(num_relations, rel_embedding_dim)
        # Transformation matrix for each relation
        self.transformation_matrices = nn.Embedding(num_relations, rel_embedding_dim * embedding_dim)

        # Initialize embeddings
        nn.init.uniform_(self.entity_embeddings.weight.data, -6/np.sqrt(embedding_dim), 6/np.sqrt(embedding_dim))
        nn.init.uniform_(self.relation_embeddings.weight.data, -6/np.sqrt(rel_embedding_dim), 6/np.sqrt(rel_embedding_dim))
        nn.init.xavier_uniform_(self.transformation_matrices.weight.data)

    def forward(self, head, relation, tail):
        # Get embeddings
        head_emb = self.entity_embeddings(head)
        tail_emb = self.entity_embeddings(tail)
        rel_emb = self.relation_embeddings(relation)
        # Get the transformation matrix for the relation
        transformation_matrix = self.transformation_matrices(relation).view(-1, self.rel_embedding_dim, self.embedding_dim)
        # Project entities into relation-specific space
        head_proj = torch.bmm(transformation_matrix, head_emb.unsqueeze(-1)).squeeze(-1)
        tail_proj = torch.bmm(transformation_matrix, tail_emb.unsqueeze(-1)).squeeze(-1)
        # Calculate score
        score = torch.norm(head_proj + rel_emb - tail_proj, p=1, dim=1)
        return score

    def get_embeddings(self):
        # Return entity and relation embeddings
        return self.entity_embeddings.weight, self.relation_embeddings.weight


Defining the TransR model class


## 5. Bernoulli Negative Sampling

In [14]:
print("Bernoulli Negative Sampling")
def bernoulli_negative_sample(triples, num_entities, num_samples=1):
    # This function generates negative samples by randomly corrupting heads or tails
    negative_samples = []
    for head, relation, tail in triples:
        for _ in range(num_samples):
            if np.random.rand() < 0.5:  # Replace head with random entity
                neg_head = np.random.randint(0, num_entities)
                while neg_head == head:
                    neg_head = np.random.randint(0, num_entities)
                negative_samples.append((neg_head, relation, tail))
            else:  # Replace tail with random entity
                neg_tail = np.random.randint(0, num_entities)
                while neg_tail == tail:
                    neg_tail = np.random.randint(0, num_entities)
                negative_samples.append((head, relation, neg_tail))
    return negative_samples

In [15]:
print("Margin Ranking Loss")
def margin_ranking_loss(pos_score, neg_score, margin):
    return torch.mean(torch.clamp(margin + pos_score - neg_score, min=0))

## 6. Training

In [16]:
# Instantiate models and define training parameters
embedding_dim = 100
rel_embedding_dim = 100  # For TransR
margin_values = [1.0, 2.0, 3.0, 4.0, 5.0]  # Varying margin values
learning_rate = 0.001
num_epochs = 50

print("Models initialized with embedding dimension 100 and margin 1.0.")

# Sample initialization for TransE and TransR with a sample margin
transe_model = TransE(nations_num_entities, nations_num_relations, embedding_dim, margin=1.0).to(device)
transr_model = TransR(nations_num_entities, nations_num_relations, embedding_dim, margin=1.0, rel_embedding_dim=rel_embedding_dim).to(device)

# Define optimizers
optimizer_transe = optim.Adam(transe_model.parameters(), lr=learning_rate)
optimizer_transr = optim.Adam(transr_model.parameters(), lr=learning_rate)


Models initialized with embedding dimension 100 and margin 1.0.


In [17]:
# Training function for TransE and TransR with Bernoulli Negative Sampling
def train_model(model, optimizer, criterion, train_triples, num_entities, batch_size, num_epochs, device, margin):
    model.train()
    for epoch in range(num_epochs):
        np.random.shuffle(train_triples)  # Shuffle triples for each epoch
        total_loss = 0

        # Batch training
        for i in range(0, len(train_triples), batch_size):
            batch_triples = train_triples[i:i + batch_size]
            pos_head, pos_rel, pos_tail = zip(*batch_triples)

            # Convert to torch tensors
            pos_head = torch.tensor(pos_head).to(device)
            pos_rel = torch.tensor(pos_rel).to(device)
            pos_tail = torch.tensor(pos_tail).to(device)

            # Generate negative samples using Bernoulli Negative Sampling
            neg_triples = bernoulli_negative_sample(batch_triples, num_entities)
            neg_head, neg_rel, neg_tail = zip(*neg_triples)
            neg_head = torch.tensor(neg_head).to(device)
            neg_rel = torch.tensor(neg_rel).to(device)
            neg_tail = torch.tensor(neg_tail).to(device)

            # Calculate scores for positive and negative samples
            pos_score = model(pos_head, pos_rel, pos_tail)
            neg_score = model(neg_head, neg_rel, neg_tail)

            # Calculate loss
            loss = criterion(pos_score, neg_score, margin)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

## 7. Evaluation Function

In [18]:
# Evaluation function for 1-hop question-answering
from pykeen.evaluation import RankBasedEvaluator

def evaluate_model(model, test_triples, device, num_entities):
    model.eval()
    evaluator = RankBasedEvaluator()
    metrics = {'mean_rank': [], 'hits_at_10': []}

    with torch.no_grad():
        for head, rel, tail in test_triples:
            # Head prediction
            head_predictions = []
            for candidate in range(num_entities):
                score = model(torch.tensor([candidate]).to(device),
                              torch.tensor([rel]).to(device),
                              torch.tensor([tail]).to(device))
                head_predictions.append((candidate, score.item()))
            head_predictions = sorted(head_predictions, key=lambda x: x[1])
            correct_head_rank = next(i for i, (entity, _) in enumerate(head_predictions) if entity == head)
            metrics['mean_rank'].append(correct_head_rank + 1)
            metrics['hits_at_10'].append(1 if correct_head_rank < 10 else 0)

            # Tail prediction
            tail_predictions = []
            for candidate in range(num_entities):
                score = model(torch.tensor([head]).to(device),
                              torch.tensor([rel]).to(device),
                              torch.tensor([candidate]).to(device))
                tail_predictions.append((candidate, score.item()))
            tail_predictions = sorted(tail_predictions, key=lambda x: x[1])
            correct_tail_rank = next(i for i, (entity, _) in enumerate(tail_predictions) if entity == tail)
            metrics['mean_rank'].append(correct_tail_rank + 1)
            metrics['hits_at_10'].append(1 if correct_tail_rank < 10 else 0)

    # Calculate Mean Rank and Hits@10
    mean_rank = np.mean(metrics['mean_rank'])
    hits_at_10 = np.mean(metrics['hits_at_10'])

    print(f"Evaluation Results: Mean Rank (MR) = {mean_rank}, Hits@10 = {hits_at_10 * 100}%")


## 8. Execution

In [19]:
# Initialize model, criterion, and train TransE with margin 1.0 as a demonstration
margin = 1.0
criterion = margin_ranking_loss

In [20]:
train_model(transe_model, optimizer_transe, criterion, nations_train_triples, nations_num_entities, batch_size=128, num_epochs=num_epochs, device=device, margin=margin)
# Example evaluation call on Nations dataset test triples
print("Evaluating TransE model...")
evaluate_model(transe_model, nations_test_triples, device, nations_num_entities)

Epoch [1/50], Loss: 45.4096
Epoch [2/50], Loss: 42.9939
Epoch [3/50], Loss: 42.3667
Epoch [4/50], Loss: 40.3386
Epoch [5/50], Loss: 38.3300
Epoch [6/50], Loss: 39.7635
Epoch [7/50], Loss: 34.2934
Epoch [8/50], Loss: 36.5350
Epoch [9/50], Loss: 32.3080
Epoch [10/50], Loss: 33.4884
Epoch [11/50], Loss: 34.5222
Epoch [12/50], Loss: 32.1814
Epoch [13/50], Loss: 30.2607
Epoch [14/50], Loss: 30.6974
Epoch [15/50], Loss: 30.4464
Epoch [16/50], Loss: 29.4711
Epoch [17/50], Loss: 28.0881
Epoch [18/50], Loss: 26.9708
Epoch [19/50], Loss: 24.7332
Epoch [20/50], Loss: 27.7638
Epoch [21/50], Loss: 28.8775
Epoch [22/50], Loss: 25.7822
Epoch [23/50], Loss: 26.3975
Epoch [24/50], Loss: 25.4755
Epoch [25/50], Loss: 26.9676
Epoch [26/50], Loss: 25.5370
Epoch [27/50], Loss: 25.5878
Epoch [28/50], Loss: 23.7035
Epoch [29/50], Loss: 23.8597
Epoch [30/50], Loss: 21.6219
Epoch [31/50], Loss: 21.8558
Epoch [32/50], Loss: 22.1228
Epoch [33/50], Loss: 21.6265
Epoch [34/50], Loss: 22.0621
Epoch [35/50], Loss: 23

In [21]:
# Example training call for TransR model on Nations dataset with margin 1.0
train_model(transr_model, optimizer_transr, criterion, nations_train_triples, nations_num_entities, batch_size=128, num_epochs=num_epochs, device=device, margin=margin)
# Evaluation for TransR
print("Evaluating TransR model...")
evaluate_model(transr_model, nations_test_triples, device, nations_num_entities)


Epoch [1/50], Loss: 20.0288
Epoch [2/50], Loss: 18.6452
Epoch [3/50], Loss: 18.3549
Epoch [4/50], Loss: 15.7413
Epoch [5/50], Loss: 16.4710
Epoch [6/50], Loss: 15.4581
Epoch [7/50], Loss: 14.8644
Epoch [8/50], Loss: 14.7310
Epoch [9/50], Loss: 13.9629
Epoch [10/50], Loss: 13.5664
Epoch [11/50], Loss: 12.4545
Epoch [12/50], Loss: 12.7124
Epoch [13/50], Loss: 12.8867
Epoch [14/50], Loss: 12.0041
Epoch [15/50], Loss: 12.2193
Epoch [16/50], Loss: 12.7536
Epoch [17/50], Loss: 10.8840
Epoch [18/50], Loss: 11.2657
Epoch [19/50], Loss: 11.3740
Epoch [20/50], Loss: 12.1895
Epoch [21/50], Loss: 12.0831
Epoch [22/50], Loss: 11.5057
Epoch [23/50], Loss: 10.7513
Epoch [24/50], Loss: 10.6494
Epoch [25/50], Loss: 11.3159
Epoch [26/50], Loss: 10.5647
Epoch [27/50], Loss: 10.3132
Epoch [28/50], Loss: 10.3277
Epoch [29/50], Loss: 10.8625
Epoch [30/50], Loss: 11.0124
Epoch [31/50], Loss: 10.9242
Epoch [32/50], Loss: 10.2241
Epoch [33/50], Loss: 10.7497
Epoch [34/50], Loss: 10.2484
Epoch [35/50], Loss: 9.

## 9. Similar Fact Retrieval Task

In [22]:
import torch
import torch.nn as nn
import numpy as np

In [23]:
print("Some declartion and embeddings")
validation_triples = [
    ('brazil', 'commonbloc1', 'india'),
    ('burma', 'intergovorgs3', 'indonesia'),
    ('china', 'accusation', 'uk'),
    ('cuba', 'reldiplomacy', 'china'),
    ('egypt', 'embassy', 'uk')
]

embedding_dim = 30
rel_embedding_dim = 30
transe_model = TransE(nations_num_entities, nations_num_relations, embedding_dim, margin=1.0).to(device)
transr_model = TransR(nations_num_entities, nations_num_relations, embedding_dim, margin=1.0, rel_embedding_dim=rel_embedding_dim).to(device)

Some declartion and embeddings


In [24]:
print('COnverting nations entity to id')
def entity_to_id_nations(nations_dataset):
  entity_to_id = {entity: entity_id for entity_id, entity in enumerate(nations_dataset.entity_to_id)}
  return entity_to_id

print('COnverting nations relation to id')
def relation_to_id_nations(nations_dataset):
  relation_to_id = {relation: relation_id for relation_id, relation in enumerate(nations_dataset.relation_to_id)}
  return relation_to_id

nations_entity_to_id = entity_to_id_nations(nations_dataset)
nations_relation_to_id = relation_to_id_nations(nations_dataset)

COnverting nations entity to id
COnverting nations relation to id


In [25]:
# Validating is id generated propelry
print("nations_entity_to_id:",nations_entity_to_id)
print("nations_relation_to_id:",nations_relation_to_id)

nations_entity_to_id: {'brazil': 0, 'burma': 1, 'china': 2, 'cuba': 3, 'egypt': 4, 'india': 5, 'indonesia': 6, 'israel': 7, 'jordan': 8, 'netherlands': 9, 'poland': 10, 'uk': 11, 'usa': 12, 'ussr': 13}
nations_relation_to_id: {'accusation': 0, 'aidenemy': 1, 'attackembassy': 2, 'blockpositionindex': 3, 'booktranslations': 4, 'boycottembargo': 5, 'commonbloc0': 6, 'commonbloc1': 7, 'commonbloc2': 8, 'conferences': 9, 'dependent': 10, 'duration': 11, 'economicaid': 12, 'eemigrants': 13, 'embassy': 14, 'emigrants3': 15, 'expeldiplomats': 16, 'exportbooks': 17, 'exports3': 18, 'independence': 19, 'intergovorgs': 20, 'intergovorgs3': 21, 'lostterritory': 22, 'militaryactions': 23, 'militaryalliance': 24, 'negativebehavior': 25, 'negativecomm': 26, 'ngo': 27, 'ngoorgs3': 28, 'nonviolentbehavior': 29, 'officialvisits': 30, 'pprotests': 31, 'relbooktranslations': 32, 'reldiplomacy': 33, 'releconomicaid': 34, 'relemigrants': 35, 'relexportbooks': 36, 'relexports': 37, 'relintergovorgs': 38, 're

In [26]:
print("Retrieve all triples from the Nations dataset.")
def get_all_triples(nations_dataset):
    all_triples = []
    for triple in nations_dataset.training.mapped_triples.tolist():
        head_id, rel_id, tail_id = triple
        head = list(nations_entity_to_id.keys())[list(nations_entity_to_id.values()).index(head_id)]
        relation = list(nations_relation_to_id.keys())[list(nations_relation_to_id.values()).index(rel_id)]
        tail = list(nations_entity_to_id.keys())[list(nations_entity_to_id.values()).index(tail_id)]
        all_triples.append((head, relation, tail))
    return all_triples

print("Retrieve similar triples for each validation triple.")
def retrieve_similar_triples(model, validation_triples, all_triples, entity_to_id, relation_to_id, k=5):
    similar_triples = {}
    model.eval()

    with torch.no_grad():
        for val_triple in validation_triples:
            print(f"Processing validation triple: {val_triple}")
            val_triple_emb = get_triple_embedding(model, val_triple, entity_to_id, relation_to_id)
            scores = []

            for train_triple in all_triples:
                if train_triple == tuple(val_triple):
                    continue

                train_triple_emb = get_triple_embedding(model, train_triple, entity_to_id, relation_to_id)
                if val_triple_emb is not None and train_triple_emb is not None:
                    score = calculate_similarity_score(val_triple_emb, train_triple_emb)
                    scores.append((train_triple, score))

            top_k_similar = sorted(scores, key=lambda x: x[1], reverse=True)[:k]
            similar_triples[tuple(val_triple)] = top_k_similar

    return similar_triples

print("Get the embedding of a triple using the given model.")
def get_triple_embedding(model, triple, entity_to_id, relation_to_id):
    head, relation, tail = triple

    try:
        head_id = entity_to_id[head]
        rel_id = relation_to_id[relation]
        tail_id = entity_to_id[tail]
    except KeyError as e:
        print(f"KeyError for triple {triple}: {e}")
        return None

    head_id = torch.tensor([head_id]).to(device)
    rel_id = torch.tensor([rel_id]).to(device)
    tail_id = torch.tensor([tail_id]).to(device)

    if isinstance(model, TransR):
        # Get embeddings
        head_emb = model.entity_embeddings(head_id)
        tail_emb = model.entity_embeddings(tail_id)
        rel_emb = model.relation_embeddings(rel_id)

        # Get transformation matrix
        transformation_matrix = model.transformation_matrices(rel_id).view(
            -1, model.rel_embedding_dim, model.embedding_dim)

        # Project entities into relation space
        head_proj = torch.bmm(transformation_matrix, head_emb.unsqueeze(-1)).squeeze(-1)
        tail_proj = torch.bmm(transformation_matrix, tail_emb.unsqueeze(-1)).squeeze(-1)

        # Combine embeddings in relation space
        triple_embedding = torch.cat([head_proj.squeeze(0), rel_emb.squeeze(0), tail_proj.squeeze(0)])
    # STarting code for TransE
    else:
        head_emb = model.entity_embeddings(head_id)
        rel_emb = model.relation_embeddings(rel_id)
        tail_emb = model.entity_embeddings(tail_id)
        triple_embedding = torch.cat([head_emb.squeeze(0), rel_emb.squeeze(0), tail_emb.squeeze(0)])

    return triple_embedding


Retrieve all triples from the Nations dataset.
Retrieve similar triples for each validation triple.
Get the embedding of a triple using the given model.


In [27]:
print("Calculate the dot product-based similarity score between two triple embeddings.")
def calculate_similarity_score(triple_emb_1, triple_emb_2):
    triple_emb_1_norm = triple_emb_1 / (torch.norm(triple_emb_1) + 1e-8)
    triple_emb_2_norm = triple_emb_2 / (torch.norm(triple_emb_2) + 1e-8)
    return torch.dot(triple_emb_1_norm, triple_emb_2_norm).item()


Calculate the dot product-based similarity score between two triple embeddings.


In [28]:

print("Getting all triples from the dataset...")
all_triples = get_all_triples(nations_dataset)
print(f"Total number of triples: {len(all_triples)}")


Getting all triples from the dataset...
Total number of triples: 1592


In [29]:
# TRANSE
print("\nFinding similar triples using TransE...")
similar_triples_transe = retrieve_similar_triples(
    transe_model,
    validation_triples,
    all_triples,
    nations_entity_to_id,
    nations_relation_to_id,
    k=5
)


Finding similar triples using TransE...
Processing validation triple: ('brazil', 'commonbloc1', 'india')
Processing validation triple: ('burma', 'intergovorgs3', 'indonesia')
Processing validation triple: ('china', 'accusation', 'uk')
Processing validation triple: ('cuba', 'reldiplomacy', 'china')
Processing validation triple: ('egypt', 'embassy', 'uk')


In [30]:
# TRANSR
print("\nFinding similar triples using TransR...")
similar_triples_transr = retrieve_similar_triples(
    transr_model,
    validation_triples,
    all_triples,
    nations_entity_to_id,
    nations_relation_to_id,
    k=5
)


Finding similar triples using TransR...
Processing validation triple: ('brazil', 'commonbloc1', 'india')
Processing validation triple: ('burma', 'intergovorgs3', 'indonesia')
Processing validation triple: ('china', 'accusation', 'uk')
Processing validation triple: ('cuba', 'reldiplomacy', 'china')
Processing validation triple: ('egypt', 'embassy', 'uk')


In [31]:
print("\nTop 5 similar triples for each validation triple using TransE:")
for val_triple, sim_triples in similar_triples_transe.items():
    print(f"\nValidation triple: {val_triple}")
    for triple, score in sim_triples:
        print(f"Similar triple: {triple}, Similarity score: {score:.4f}")



Top 5 similar triples for each validation triple using TransE:

Validation triple: ('brazil', 'commonbloc1', 'india')
Similar triple: ('ussr', 'commonbloc1', 'india'), Similarity score: 0.8340
Similar triple: ('brazil', 'relintergovorgs', 'india'), Similarity score: 0.7764
Similar triple: ('brazil', 'commonbloc1', 'burma'), Similarity score: 0.7740
Similar triple: ('brazil', 'ngoorgs3', 'india'), Similarity score: 0.7702
Similar triple: ('brazil', 'commonbloc1', 'egypt'), Similarity score: 0.7494

Validation triple: ('burma', 'intergovorgs3', 'indonesia')
Similar triple: ('india', 'intergovorgs3', 'indonesia'), Similarity score: 0.7536
Similar triple: ('burma', 'intergovorgs3', 'usa'), Similarity score: 0.7249
Similar triple: ('burma', 'exports3', 'indonesia'), Similarity score: 0.7104
Similar triple: ('burma', 'intergovorgs3', 'uk'), Similarity score: 0.6988
Similar triple: ('burma', 'relngo', 'indonesia'), Similarity score: 0.6881

Validation triple: ('china', 'accusation', 'uk')
Si

In [32]:

print("\nTop 5 similar triples for each validation triple using TransR:")
for val_triple, sim_triples in similar_triples_transr.items():
    print(f"\nValidation triple: {val_triple}")
    for triple, score in sim_triples:
        print(f"Similar triple: {triple}, Similarity score: {score:.4f}")


Top 5 similar triples for each validation triple using TransR:

Validation triple: ('brazil', 'commonbloc1', 'india')
Similar triple: ('poland', 'commonbloc1', 'india'), Similarity score: 0.9609
Similar triple: ('brazil', 'commonbloc1', 'burma'), Similarity score: 0.9560
Similar triple: ('brazil', 'commonbloc1', 'jordan'), Similarity score: 0.9515
Similar triple: ('usa', 'commonbloc1', 'india'), Similarity score: 0.9479
Similar triple: ('ussr', 'commonbloc1', 'india'), Similarity score: 0.9450

Validation triple: ('burma', 'intergovorgs3', 'indonesia')
Similar triple: ('india', 'intergovorgs3', 'indonesia'), Similarity score: 0.9319
Similar triple: ('burma', 'intergovorgs3', 'india'), Similarity score: 0.9312
Similar triple: ('burma', 'intergovorgs3', 'brazil'), Similarity score: 0.9286
Similar triple: ('burma', 'intergovorgs3', 'uk'), Similarity score: 0.9222
Similar triple: ('burma', 'intergovorgs3', 'usa'), Similarity score: 0.9130

Validation triple: ('china', 'accusation', 'uk')
